## Define Model

In [1]:
import matplotlib
from keras import backend as K
K.set_image_dim_ordering('th')

/home/callum/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
Using cuDNN version 7005 on context None
Preallocating 4861/6077 Mb (0.800000) on cuda0
Mapped name None to device cuda0: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
from keras.models import Sequential
from keras.layers import ZeroPadding2D, Conv2D
from keras.layers import MaxPooling2D, BatchNormalization
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def ConvBlock(layers, filters):
        for i in range(layers):
            model.add(ZeroPadding2D((1, 1)))
            model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu'))  # Keras2
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))
def FCBlock():
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.6))
        #model.add(BatchNormalization())

In [3]:
model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
ConvBlock(2, 64)
ConvBlock(2, 128)
ConvBlock(3, 256)
ConvBlock(3, 512)
ConvBlock(3, 512)

model.add(Flatten())
FCBlock()
FCBlock()
model.add(Dense(1000, activation='softmax'))
model.load_weights('fastai/models/vgg16.h5')
for layer in model.layers: layer.trainable=False
model.pop()
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('models/dogscats_ft_preprocessed2.h5')

In [12]:
Dropout_indxs = [index for index,layer in enumerate(model.layers) 
                     if type(layer) is Dropout][::-1]
fake_model = Sequential(model.layers[:35])
#fake_model.add(BatchNormalization())
fake_model.add(model.layers[35])
fake_model.add(model.layers[36])
#fake_model.add(BatchNormalization())
fake_model.add(model.layers[-1])
for layer in fake_model.layers[33:]:
    layer.trainable=True
#model.optimizer.lr.set_value(0.0001)
fake_model.compile(optimizer=RMSprop(lr=.00001, rho=.7), loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('models/dogscats_ft_preprocessed2.h5')

In [13]:
conv_layers = model.layers[:31]
conv_model = Sequential(conv_layers)
fc_layers = model.layers[31:]

### Import the data

In [14]:
path='data/dogscats'
train_path=path+"/train"
valid_path=path+"/valid"

In [15]:
#Get data
from keras.preprocessing import image

gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=.1, height_shift_range=.1,
                              shear_range=.15, zoom_range=.1, channel_shift_range=10.,
                              horizontal_flip=True)
batch_size=24

train_batches = gen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)
val_batches = gen.flow_from_directory(valid_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)

classes = list(iter(train_batches.class_indices)) # get a list of all the class labels

for c in train_batches.class_indices:
    classes[train_batches.class_indices[c]] = c

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [16]:
steps_per_epoch = int(np.ceil(train_batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/batch_size))

# trn_classes = train_batches.classes
# val_classes = val_batches.classes
# val_features = conv_model.predict_generator(val_batches, validation_steps)
# trn_features = conv_model.predict_generator(train_batches, steps_per_epoch)
# trn_features.shape

## Fit the model

In [17]:
#for layer in fake_model.layers[33:]:
#    layer.trainable=True
#model.optimizer.lr.set_value(0.0001)


In [18]:
# Fit FC Layers
#fake_model.load_weights('models/dogscats2.h5')
#fc_model.fit(trn_features, trn_labels, epochs=1, batch_size=batch_size, validation_data=(val_features, val_labels))

In [19]:
#Fit Model
#model.load_weights('models/dogscats2.h5')

fake_model.fit_generator(train_batches, steps_per_epoch=int(np.ceil(train_batches.samples/batch_size)), epochs=1,
                validation_data=val_batches, validation_steps=int(np.ceil(val_batches.samples/batch_size)))
#model.fit_generator(train_batches, samples_per_epoch=train_batches.samples, epochs=1, validation_data=val_batches, validation_steps=val_batches.samples)

Epoch 1/1
959/958 [==============================] - 280s 292ms/step - loss: 0.0982 - acc: 0.9900 - val_loss: 0.1648 - val_acc: 0.9845


In [20]:
fake_model.save_weights('models/dogscats_ft_preprocessed2.h5')

## Examine Results

In [ ]:
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
import matplotlib.pyplot as plt
#Use the plots helper function
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')



In [ ]:
sample_batches = gen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=4)

In [ ]:
# Get a few images and their true labels
imgs, labels = next(sample_batches)
indxs = np.argmax(labels, axis=1)
indxs
labels = [classes[indx] for indx in indxs]
true_labels_dict = {
    'c0': 'safe driving',
    'c1': 'texting - right',
    'c2': 'talking on the phone - right',
    'c3': 'texting - left',
    'c4': 'talking on the phone - left',
    'c5': 'operating the radio',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'hair and makeup',
    'c9': 'talking to passenger',
}
true_labels = [true_labels_dict[label] for label in labels]

In [ ]:
plots(imgs, titles=true_labels)

## Test Model

In [44]:
#fake_model.load_weights('models/dogscats_ft_preprocessed2.h5')
test_batches = gen.flow_from_directory(path+'/test', target_size=(224,224), class_mode='categorical', shuffle=False, batch_size=20)
test_steps = int(np.ceil(test_batches.samples/batch_size))
ids = []
for filename in test_batches.filenames:
    ids.append(filename.split('/')[1].split('.')[0])
preds = fake_model.predict_generator(test_batches, test_steps)
#ids = []


Found 12500 images belonging to 1 classes.


['1',
 '10',
 '100',
 '1000',
 '10000',
 '10001',
 '10002',
 '10003',
 '10004',
 '10005',
 '10006',
 '10007',
 '10008',
 '10009',
 '1001',
 '10010',
 '10011',
 '10012',
 '10013',
 '10014',
 '10015',
 '10016',
 '10017',
 '10018',
 '10019',
 '1002',
 '10020',
 '10021',
 '10022',
 '10023',
 '10024',
 '10025',
 '10026',
 '10027',
 '10028',
 '10029',
 '1003',
 '10030',
 '10031',
 '10032',
 '10033',
 '10034',
 '10035',
 '10036',
 '10037',
 '10038',
 '10039',
 '1004',
 '10040',
 '10041',
 '10042',
 '10043',
 '10044',
 '10045',
 '10046',
 '10047',
 '10048',
 '10049',
 '1005',
 '10050',
 '10051',
 '10052',
 '10053',
 '10054',
 '10055',
 '10056',
 '10057',
 '10058',
 '10059',
 '1006',
 '10060',
 '10061',
 '10062',
 '10063',
 '10064',
 '10065',
 '10066',
 '10067',
 '10068',
 '10069',
 '1007',
 '10070',
 '10071',
 '10072',
 '10073',
 '10074',
 '10075',
 '10076',
 '10077',
 '10078',
 '10079',
 '1008',
 '10080',
 '10081',
 '10082',
 '10083',
 '10084',
 '10085',
 '10086',
 '10087',
 '10088',
 '10089'

In [66]:
import pandas as pd
df = pd.DataFrame()
df['id'] = ids
df['label'] = np.clip(preds[:, 1], 0.05, 0.95)
df[['id', 'label']]

,id,label
0,1,0.95
1,10,0.05
2,100,0.05
3,1000,0.95
4,10000,0.95
5,10001,0.05
6,10002,0.05
7,10003,0.95
8,10004,0.95
9,10005,0.05


In [67]:
df.to_csv('dogscats_preds.csv', index=False)